In [21]:
import json
import numpy as np
# import quaternion
import math
import os
import cv2
import open3d as o3d
import natsort 
import shutil
from pyquaternion import Quaternion

# from PIL import Image
# from pyquaternion import Quaternion
# from scipy.spatial.transform import Rotation as R


output_path='/home/eslim/data/indoor/custom/'

asset_path ='/home/eslim/data/indoor/asset/'
label_path ='/home/eslim/data/indoor/labels/'


train_ratio = 0.7
class_names=[]

# img = cv2.imread(asset_path+'0a10c009-dc8b-4479-80f4-49db9e60b473/image_00000001_00000001.jpg')
# img_height,img_width = img.shape[:2]

# class_name_arr=np.array([])
#Q : FOV아니고 whole scene label?


In [22]:
# !pip install open3d
# !pip install natsort

In [23]:
# !python -m pip install --upgrade --force-reinstall numpy-quaternion
# !conda install -c conda-forge quaternion
#https://quaternion.readthedocs.io/en/latest/
#!pip install natsort

In [24]:
points = np.fromfile("/home/eslim/data/indoor/asset/00aa9c72-e978-4fcb-8db3-6f2531a37200/frame_00000022.bin", dtype=np.float32).reshape(-1,4)
# points[3]
# pcd = o3d.t.geometry.PointCloud()
# pcd.point = points[:3]
# points = np.fromefile("/nms/")
print(points)
print(points[5000])
np.shape(points)
points = points[:,:3]
np.shape(points)
pc = o3d.geometry.PointCloud()
pc.points = o3d.utility.Vector3dVector(points)
# # print(pc)
o3d.io.write_point_cloud("./test2.pcd", pc)


[[-9.2735014e+00  6.3342840e-04 -2.1530333e-01  0.0000000e+00]
 [-3.1894591e+00  4.4871704e-04 -1.2454424e+00  0.0000000e+00]
 [-9.2679996e+00  1.9747308e-03  0.0000000e+00  0.0000000e+00]
 ...
 [-3.2751572e+00 -6.8247184e-04 -1.2789063e+00  0.0000000e+00]
 [-9.2799997e+00 -1.2620434e-03  0.0000000e+00  0.0000000e+00]
 [-3.3077180e+00 -2.1041639e-04 -1.2039109e+00  0.0000000e+00]]
[-4.422363   6.329202   0.5399151  0.       ]


True

In [25]:
def convert_pcd_to_npy(file_path,save_path):
    point_cloud = o3d.read_point_cloud(file_path)
    pointcloud = np.asarray(point_cloud.points)[:, :3]
    zeros = np.zeros(np.shape(pointcloud)[0])
    zeros = np.reshape(zeros, (-1, 1))
    points = np.concatenate([pointcloud, zeros], axis=1)
    np.save(save_path, points)

def convert_bin_to_npy(file_path,save_path):
#     print(file_path)
    try:
        points = np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)
        np.save(save_path, points)
    except:
        print(file_path)
    

In [26]:
'''
move pcd file to each folder and rename as order
'''
def copy_pcd(mode,path,output_path,idx):       
    try:
        if not os.path.exists(output_path+'points/'):
            os.makedirs(output_path+'points/')
    except OSError:
        print("Error: Failed to create the directory.")
   
    if mode == "pcd":
        output_img_path = output_path+'points/'
        convert_pcd_to_npy(path, output_img_path+str(idx).zfill(6)+".npy")  
    elif mode == "bin":
        output_img_path = output_path+'points/'    
        convert_bin_to_npy(path, output_img_path+str(idx).zfill(6)+".npy")          



In [27]:
# try:
#     file = open(label_path+'00aa9c72-e978-4fcb-8db3-6f2531a37200.json')
# except:
#     print("Error: Failed to open the file")


# jsonString = json.load(file) 
# jsonArray = jsonString.get('objects')
# for iter_num, list in enumerate(jsonArray):
#     frames = list.get("frames")
#     class_name = list.get('class_name')
#     global class_names
#     if (class_name in class_names) is not True:

#         class_names.append(class_name)

#     for frame in frames:
#         idx = frame.get('num')

#         ann = frame.get('annotation').get('coord')
#         height = np.round_(ann.get('size').get('z'),5)
#         width = np.round_(ann.get('size').get('y'),5)
#         length = np.round_(ann.get('size').get('x'),5)
#         # TODO : check coordinate , lidar or cam? -> cam



#         x = np.round_(ann.get('position').get('x'),5)
#         y = np.round_(ann.get('position').get('y'),5)
#         z = np.round_(ann.get('position').get('z'),5) 

# #             yaw_pitch_roll = Quaternion(ann.get('rotation_quaternion').get('w'),ann.get('rotation_quaternion').get('x'),ann.get('rotation_quaternion').get('y'),ann.get('rotation_quaternion').get('z')).yaw_pitch_roll
#         yaw_pitch_roll = Quaternion(
#             ann.get('rotation_quaternion').get('w'),
#             ann.get('rotation_quaternion').get('x'),
#             ann.get('rotation_quaternion').get('y'),
#             ann.get('rotation_quaternion').get('z'),
#         ).yaw_pitch_roll
#         yaw = np.round_(yaw_pitch_roll[0], 5)
#         print(x,y,z,yaw)

In [28]:
'''
convert label info to custom form

format: [x y z dx dy dz heading_angle category_name]

'''
def preprocess_label(label_output,frame_num):
    label_output = label_output+"labels/"
    try:
        if not os.path.exists(label_output):
            os.makedirs(label_output)
    except OSError:
        print("Error: Failed to create the directory.")
    for i in range(frame_num):
        try:
            f = open(label_output+str(label_idx+i).zfill(6)+".txt", 'w')
            f.close()
        except:
            print("Fail to create file")

def convert_label_suite2custom(label_path,label_output, label_idx): 
    label_output = label_output+"labels/"
    try:
        file = open(label_path)
    except:
        print("Error: Failed to open the file")


    jsonString = json.load(file)   
    # frame_num = jsonString.get("label").get('asset').get("info").get("frame_count")


    try:
        jsonArray = jsonString.get('objects')
    except:
        print("can't find objects")
        # return label_idx
        return
    
    # calib_path = output_path+'training/calib/'
    for iter_num, list in enumerate(jsonArray):
        frames = list.get("frames")
        class_name = list.get('class_name')
        if class_name == "Wall" or class_name == 'Undefined Object' or class_name =='Undefined Stuff':
            continue
        if class_name == 'Traffic Cone':
            class_name = 'Traffic_Cone'
        global class_names
        if (class_name in class_names) is not True:
            
            class_names.append(class_name)
            
        for frame in frames:
            idx = frame.get('num')
            
            ann = frame.get('annotation').get('coord')
            height = np.round_(ann.get('size').get('z'),5)
            width = np.round_(ann.get('size').get('y'),5)
            length = np.round_(ann.get('size').get('x'),5)
            # TODO : check coordinate , lidar or cam? -> cam
            
            
            
            x = np.round_(ann.get('position').get('x'),5)
            y = np.round_(ann.get('position').get('y'),5)
            z = np.round_(ann.get('position').get('z'),5) 
           
#             yaw_pitch_roll = Quaternion(ann.get('rotation_quaternion').get('w'),ann.get('rotation_quaternion').get('x'),ann.get('rotation_quaternion').get('y'),ann.get('rotation_quaternion').get('z')).yaw_pitch_roll
            yaw_pitch_roll = Quaternion(
                ann.get('rotation_quaternion').get('w'),
                ann.get('rotation_quaternion').get('x'),
                ann.get('rotation_quaternion').get('y'),
                ann.get('rotation_quaternion').get('z'),
            ).yaw_pitch_roll
            yaw = np.round_(yaw_pitch_roll[0], 5)
#             yaw = np.round_(yaw_pitch_roll[0],5)

            f = open(label_output+str(label_idx+idx).zfill(6)+".txt", 'a')
            f.write(f'{x} {y} {z} {length} {width} {height} {yaw} {class_name}\n')
            f.close()
    return
    # return label_idx

                           

In [29]:
###calib, file movement exec###

asset_path_list = os.listdir(asset_path)
asset_path_list = natsort.natsorted(asset_path_list)




# print(asset_path_list)
pcd_idx=0
num_frame=[]
num_frame.append(0)
for data_cnt,path in enumerate(asset_path_list):
    
    files = os.listdir(asset_path+path)
    files = natsort.natsorted(files)
            
    for file in files:        
                   
        if '.bin' in file:
            #TODO : preprocession
            pcd_file_path = os.path.join(asset_path+path, file)
            copy_pcd('bin',pcd_file_path,output_path,pcd_idx)
            pcd_idx+=1
            # total_num_frame+=1
            
        if '.pcd' in file:
            pcd_file_path = os.path.join(asset_path+path, file)
            copy_pcd('pcd',pcd_file_path,output_path,pcd_idx)
            pcd_idx+=1
            # total_num_frame+=1
            
    num_frame.append(pcd_idx)
# print(f'pcd num : {pcd_idx}')



In [30]:
# num_frame
num_frame[-1]

90

In [31]:
#####label exec

label_list = os.listdir(label_path)
label_list = natsort.natsorted(label_list)

label_idx=0

preprocess_label(output_path,num_frame[-1])
for label_data_cnt,label in enumerate(label_list):
    print(label_data_cnt)
#     files = os.listdir(label_path+path)
#     for file in files:
#         if 'info.json' in file:
    json_file_path = os.path.join(label_path, label)
    file = open(json_file_path)
    jsonString = json.load(file)
    convert_label_suite2custom(json_file_path,output_path, num_frame[label_data_cnt])




0


In [32]:
#### split train, test

try:
    if not os.path.exists(output_path+'ImageSets/'):
        os.makedirs(output_path+'ImageSets/')
except OSError:
    print("Error: Failed to create the directory.")
    
f2 = open(output_path+'ImageSets/'+'train'+".txt", 'w')
f3 = open(output_path+'ImageSets/'+'val'+".txt", 'w')
for idx in range(num_frame[-1]):
    if idx < num_frame[-1]*train_ratio:
        f2.write(f'{str(idx).zfill(6)}\n')
    else:
        f3.write(f'{str(idx).zfill(6)}\n')
    idx+=1
f2.close()
f3.close()


In [33]:
class_names

['Pillar', 'Vehicle', 'Traffic_Cone', 'Human']